# Notebook 2: Preprocesamiento y Construcción de Características

Este notebook demuestra el uso de los módulos de `src/features` para:
1. Aplicar transformaciones manuales (imputación, escalado, one-hot encoding).
2. Construir nuevas características (ingeniería de características).

In [ ]:
import sys
import os
import pandas as pd
import numpy as np

# Añadir la carpeta 'src' al path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from data.loader import cargar_csv_crudo, guardar_df_procesado
from features.transformers_manual import ImputadorNumericoMediana, ImputadorCategoricoModa, EscaladorEstandarManual, OneHotManual
from features.build_features import construir_caracteristicas

## 1. Cargar Datos Crudos

Cargamos el conjunto de datos de entrenamiento para aplicar las transformaciones.

In [ ]:
RUTA_DATOS_ENTRENAMIENTO = '../data/raw/datos_entrenamiento_riesgo.csv'
df = cargar_csv_crudo(os.path.basename(RUTA_DATOS_ENTRENAMIENTO))

## 2. Pipeline de Preprocesamiento Manual

Aquí definimos un pipeline de preprocesamiento paso a paso.

In [ ]:
# Separar características y objetivo
X = df.drop('nivel_riesgo', axis=1)
y = df['nivel_riesgo']

# Identificar tipos de columnas
columnas_numericas = X.select_dtypes(include=np.number).columns.tolist()
columnas_categoricas = X.select_dtypes(include='object').columns.tolist()

# a) Imputación de nulos
imputador_num = ImputadorNumericoMediana(variables=columnas_numericas)
imputador_cat = ImputadorCategoricoModa(variables=columnas_categoricas)

imputador_num.fit(X)
X_imputado = imputador_num.transform(X)

imputador_cat.fit(X_imputado)
X_imputado = imputador_cat.transform(X_imputado)

print("Valores nulos después de la imputación:")
print(X_imputado.isnull().sum().sum())

# b) Codificación One-Hot para categóricas
codificador_onehot = OneHotManual(variables=columnas_categoricas)
codificador_onehot.fit(X_imputado)
X_codificado = codificador_onehot.transform(X_imputado)

# c) Escalado de características numéricas restantes
columnas_numericas_finales = X_codificado.select_dtypes(include=np.number).columns.tolist()
escalador = EscaladorEstandarManual()
escalador.fit(X_codificado[columnas_numericas_finales])
X_procesado_np = escalador.transform(X_codificado[columnas_numericas_finales]).values

print(f"\nForma de los datos procesados: {X_procesado_np.shape}")

## 3. Construcción de Características

Ahora usamos el script `build_features` para generar nuevas características y guardar el resultado.

In [ ]:
RUTA_SALIDA_FEATURES = '../data/processed/X_train_con_features.csv'

construir_caracteristicas(
    ruta_entrada=RUTA_DATOS_ENTRENAMIENTO, 
    ruta_salida=RUTA_SALIDA_FEATURES
)

# Cargar y mostrar el resultado
df_con_features = pd.read_csv(RUTA_SALIDA_FEATURES)
print("\nDataFrame con nuevas características:")
df_con_features.head()